# Поиск границ на проекции XZ здания и конвертация в CAD-подобный формат

## Задача

Цель эксперимента - найти границы на проекции XZ здания и конвертировать их в CAD-подобный формат. Для этого необходимо наложить примитивы (rect, arc, circle) на форму так, чтобы границы совпадали.

### Этапы работы:
1. **Импорт объектов** - загрузка .obj файлов из датасета
2. **Создание проекции** - получение проекции XZ для каждого здания
3. **Препроцессинг** - подготовка данных для анализа границ
4. **Поиск границ** - детекция контуров на проекции
5. **Аппроксимация примитивами** - наложение прямоугольников, дуг и окружностей
6. **Валидация и визуализация** - проверка совпадения границ и визуализация результатов


## 1. Импорт объектов

Загрузка всех .obj файлов из датасета через state.json


In [1]:

import json
from pathlib import Path

# Path to the state.json file
state_json_path = "../dataset/auto_building_dataset/auto_building_dataset.4d71dc38198d4baa8e60d70e1db84469/artifacts/state/state.json"

# Read the state.json file
print("Reading state.json file...")
with open(state_json_path, 'r') as f:
    state_data = json.load(f)

print(f"State data loaded successfully. Type: {type(state_data)}")

# Extract all .obj file paths
obj_paths_list = []

# Function to recursively search for .obj files in the JSON structure
def find_obj_files(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if key == "relative_path" and isinstance(value, str) and value.endswith('.obj'):
                obj_paths_list.append(value)
            else:
                find_obj_files(value)
    elif isinstance(data, list):
        for item in data:
            find_obj_files(item)

# Search for .obj files
find_obj_files(state_data)

print(f"\nFound {len(obj_paths_list)} .obj files:")
# print("=" * 50)

# # Display all .obj file paths
# for i, obj_path in enumerate(obj_paths_list, 1):
#     print(f"{i:4d}. {obj_path}")

# print(f"\nTotal .obj files found: {len(obj_paths_list)}")

# Use obj_paths_list as the canonical source
obj_paths_source = obj_paths_list

# Split into buildings (root-level files) and props (inside subdirectories)
building_paths = []
props_paths = []

for p in obj_paths_source:
    if "/" in p:
        props_paths.append(p)
    else:
        building_paths.append(p)

# Deduplicate while preserving order
building_paths = list(dict.fromkeys(building_paths))
props_paths = list(dict.fromkeys(props_paths))

# Write outputs in the current directory
buildings_out = "paths_building_models.txt"
props_out = "paths_props_models.txt"

with open(buildings_out, 'w') as f:
    for p in building_paths:
        f.write(p + "\n")

with open(props_out, 'w') as f:
    for p in props_paths:
        f.write(p + "\n")

print(f"Building models: {len(building_paths)} saved to {buildings_out}")
print(f"Props models:    {len(props_paths)} saved to {props_out}")

# Preview a few examples
print("\nSample buildings (last 30 samples):")
for s in building_paths[:30]:
    print("  ", s)
# print("\nSample props:")
# for s in props_paths[:10]:
#     print("  ", s)


Reading state.json file...
State data loaded successfully. Type: <class 'dict'>

Found 1364 .obj files:
Building models: 122 saved to paths_building_models.txt
Props models:    1242 saved to paths_props_models.txt

Sample buildings (last 30 samples):
   House_Medieval_9.obj
   Venice_House_7.obj
   Tatooine_House_1.obj
   Asian_Block_1.obj
   Venice_House_2.obj
   ._House_Medieval_8.obj
   ._Castle_9.obj
   Tatooine_House_10.obj
   Modular_Roof_Demo_1.obj
   Asian_2.obj
   ._Tatooine_House_6.obj
   Tatooine_House_5.obj
   Tatooine_House_8.obj
   Castle_11.obj
   ._Tatooine_House_1.obj
   ._Venice_House_7.obj
   House_Medieval_Gate_1.obj
   House_Medieval_4.obj
   ._Venice_House_5.obj
   Castle_1.obj
   Venice_House_5.obj
   Venice_Palace_1.obj
   House_Medieval_2.obj
   ._Venice_House_2.obj
   ._Tatooine_House_7.obj
   Venice_House_4.obj
   House_Medieval_3.obj
   Tatooine_House_2.obj
   ._Sci_Fi_3.obj
   ._Venice_Palace_1.obj


## 2. Создание проекции XZ

Для каждого .obj файла необходимо:
- Загрузить 3D модель
- Создать проекцию на плоскость XZ (вид сверху)
- Получить бинарное изображение или контур проекции


In [17]:
from pathlib import Path
import numpy as np
import cv2


def read_obj_vertices_faces(obj_path):
    vertices, faces = [], []
    with open(obj_path, "r", encoding="utf-8") as f:
        for raw_line in f:
            line = raw_line.strip()
            if line.startswith("v "):
                _, x, y, z, *rest = line.split()
                vertices.append([float(x), float(y), float(z)])
            elif line.startswith("f "):
                parts = line.split()[1:]
                face = []
                for part in parts:
                    idx = part.split("/")[0]
                    if idx:
                        face.append(int(idx) - 1)
                if len(face) >= 2:
                    faces.append(face)
    return np.asarray(vertices, dtype=np.float32), faces


def project_plane(vertices, plane="xz"):
    idx_map = {"xy": (0, 1), "xz": (0, 2), "zy": (2, 1)}
    if plane not in idx_map:
        raise ValueError(f"plane must be one of {tuple(idx_map.keys())}")
    i, j = idx_map[plane]
    return vertices[:, [i, j]]


def build_edge_indices(faces):
    edges = set()
    for face in faces:
        for k in range(len(face)):
            a, b = face[k], face[(k + 1) % len(face)]
            edges.add(tuple(sorted((a, b))))
    return list(edges)


def coords_to_pixels(coords, width=1500, height=1500, padding_ratio=0.05):
    mins = coords.min(axis=0)
    maxs = coords.max(axis=0)
    spans = np.maximum(maxs - mins, 1e-9)
    usable_w = width * (1 - 2 * padding_ratio)
    usable_h = height * (1 - 2 * padding_ratio)
    scale = min(usable_w / spans[0], usable_h / spans[1])
    centered = coords - (mins + spans / 2.0)
    scaled = centered * scale
    pixels_x = width / 2.0 + scaled[:, 0]
    pixels_y = height / 2.0 - scaled[:, 1]
    return np.column_stack((pixels_x, pixels_y)).astype(np.int32)


def draw_projection_cv2(
    coords,
    edge_indices,
    plane,
    out_path,
    width=1500,
    height=1500,
    include_points=False,
    point_stride=10,
    line_color=(0, 0, 0),
    thickness=1,
):
    pixel_coords = coords_to_pixels(coords, width=width, height=height)
    canvas = np.full((height, width, 3), 255, dtype=np.uint8)

    if include_points and len(pixel_coords):
        stride = max(1, point_stride)
        for x, y in pixel_coords[::stride]:
            cv2.circle(canvas, (int(x), int(y)), 0, (190, 190, 255), -1, lineType=cv2.LINE_AA)

    if edge_indices:
        lines = [
            np.array([pixel_coords[a], pixel_coords[b]], dtype=np.int32).reshape(-1, 1, 2) for a, b in edge_indices
        ]
        cv2.polylines(canvas, lines, isClosed=False, color=line_color, thickness=thickness, lineType=cv2.LINE_AA)

    if out_path is not None:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), canvas)
        print(f"Saved {plane.upper()} projection to {out_path.resolve()}")

    return canvas


data_dir = Path(
    "../dataset/auto_building_dataset/auto_building_dataset.4d71dc38198d4baa8e60d70e1db84469/artifacts/data/"
)
for obj_path in data_dir.glob("*.obj"):
    if any(obj_path.stem.startswith(prefix) for prefix in (".", "-", "_")):
        continue
    vertices, faces = read_obj_vertices_faces(obj_path)
    print(f"Loaded {len(vertices)} vertices / {len(faces)} faces from {obj_path.name}")
    edges = build_edge_indices(faces)
    canvases = []
    for plane in ("xy", "xz", "zy"):
        coords = project_plane(vertices, plane=plane)
        canvas = draw_projection_cv2(
            coords,
            edges,
            plane=plane,
            out_path=None,
            width=1500,
            height=1500,
            include_points=False,
            thickness=1,
        )
        canvases.append(canvas)
    
    united_canvas = np.hstack(canvases)
    cv2.imwrite(f"{obj_path.stem}.png", united_canvas)


Loaded 40403 vertices / 34913 faces from Castle_8.obj
Loaded 68243 vertices / 61690 faces from Building_NeoClassical_1.obj
Loaded 15603 vertices / 15045 faces from Castle_9.obj
Loaded 16498 vertices / 14641 faces from Tatooine_House_8.obj
Loaded 27301 vertices / 25507 faces from HouseOld_3.obj
Loaded 3196 vertices / 2355 faces from Tatooine_House_9.obj
Loaded 697868 vertices / 87299 faces from House_Medieval_8.obj
Loaded 48773 vertices / 44174 faces from HouseOld_1.obj
Loaded 16520 vertices / 15164 faces from Modular_Roof_Demo_1.obj
Loaded 538292 vertices / 590256 faces from Sci_Fi_4.obj
Loaded 1222802 vertices / 488505 faces from Venice_Palace_3.obj
Loaded 228467 vertices / 282600 faces from Sci_Fi_2.obj
Loaded 120598 vertices / 124308 faces from Castle_10.obj
Loaded 549841 vertices / 520769 faces from Castle_11.obj
Loaded 293117 vertices / 23339 faces from Venice_House_3.obj
Loaded 215681 vertices / 123592 faces from Venice_House_4.obj
Loaded 223969 vertices / 195524 faces from Tatoo

## 3. Препроцессинг

Подготовка проекции для анализа:
- Нормализация координат
- Устранение шумов
- Сглаживание контуров
- Масштабирование до рабочего разрешения


In [ ]:
# TODO: Реализовать препроцессинг проекции
# - Нормализация координат в единую систему
# - Применение фильтров для устранения шумов
# - Сглаживание контуров (например, через морфологические операции)
# - Масштабирование до фиксированного разрешения


## 4. Поиск границ

Детекция контуров на проекции:
- Нахождение всех контуров (внешних и внутренних)
- Иерархия контуров (родительские и дочерние)
- Анализ геометрии контуров


In [ ]:
# TODO: Реализовать поиск границ
# - Использование алгоритмов детекции контуров (например, cv2.findContours)
# - Определение иерархии контуров
# - Анализ геометрических свойств каждого контура
# - Классификация контуров по типу (прямые линии, кривые, замкнутые фигуры)


## 5. Аппроксимация примитивами

Наложение CAD-примитивов на найденные границы:
- **Прямоугольники (rect)** - для прямых углов и прямоугольных сегментов
- **Дуги (arc)** - для криволинейных сегментов
- **Окружности (circle)** - для круглых элементов

Алгоритм должен минимизировать ошибку между реальными границами и аппроксимирующими примитивами.


In [ ]:
# TODO: Реализовать аппроксимацию примитивами
# - Для каждого сегмента контура определить тип примитива (rect/arc/circle)
# - Подобрать параметры примитива (размеры, углы, радиусы)
# - Минимизировать ошибку аппроксимации
# - Объединить примитивы в единую структуру


## 6. Валидация и визуализация

Проверка качества аппроксимации:
- Сравнение исходных границ с аппроксимированными примитивами
- Вычисление метрик точности (IoU, расстояние Хаусдорфа и т.д.)
- Визуализация результатов: исходная проекция, найденные границы, наложенные примитивы


In [ ]:
# TODO: Реализовать валидацию и визуализацию
# - Вычисление метрик качества аппроксимации
# - Визуализация исходной проекции
# - Визуализация найденных контуров
# - Визуализация наложенных примитивов (rect, arc, circle)
# - Сравнительный анализ результатов


## 7. Экспорт в CAD-подобный формат

Сохранение результатов в формате, пригодном для CAD-систем:
- Структурированное представление примитивов
- Параметры каждого примитива (координаты, размеры, углы)
- Возможность экспорта в стандартные форматы (DXF, SVG и т.д.)


In [ ]:
# TODO: Реализовать экспорт в CAD-подобный формат
# - Определить структуру данных для хранения примитивов
# - Реализовать сериализацию примитивов (JSON, XML и т.д.)
# - Опционально: экспорт в DXF, SVG или другие CAD-форматы
